# Statystyczne metody przetwarzania danych 

## Laboratorium 2 - algorytm *k* Najbliższych Sąsiadów (*k*NN)


### Opis
Celem laboratorium jest implementacja klasyfikatora *k* najbliższych sąsiadów - *k*NN (*k Nearest Neighbors*) oraz zapoznanie się z innymi metrykami klasyfikacji.

### Termin
Zadanie należy wykonać tego samego dnia. 

### System oceniania

| Liczba punktów (procentowo) | Ocena |
| :----                    | ---: |
| [0-50)   | 2   |
| [50-60)  | 3   |
| [60-70)  | 3.5 |
| [70-80)  | 4   |
| [80-90)  | 4.5 |
| [90-100] | 5   |

<u>Punkty ujemne</u>

* `ocena - 0.5` jeżeli zadanie wysłano po laboratorium, ale < 7 dni; 
* `ocena - 1` jeżeli zadanie wysłano w terminie pomiędzy 7 a 14 dni;
* `ocena - 1.5` jeżeli zadanie wysłano po upływie 14 dni, ale przed ostatnim laboratorium;
* `ocena = 2` jeżeli zadanie wysłano po ostatnim laboratorium.

<u>Uwaga:</u>

Niedopuszczalne jest dzielenie się notatnikiem (plik `.ipynb`) z innymi studentami ani udostępnianie go w Internecie. Każdy student powinien pobrać notatnik samodzielnie z platformy WIKAMP.
Wysyłając zadanie potwierdasz, że wykonałeś je samodzielnie i jest to Twoja indywidualna praca a materiał przedstawiony w tej pracy jest dla Ciebie zrozumiały. Prace bardzo podobne albo grupowe będą uznawane za plagiat.


### Zbiór danych

Zbiór danych znajduje się w katalogu `dataset/*`. Jest to zmodyfikowany zbiór danych znajdujący się pod adresem: <https://archive.ics.uci.edu/ml/datasets/leaf>.

### Przesyłanie zadań

Wszystkie pliki należy spakować archiwizatorem **zip** i przesłać za pośrednictwem platformy WIKAMP. Poniżej oczekiwana zawartość archiwum:

```
+-- 📂 [IMIE.NAZWISKO].zip
    +-- 📜 Lab02.ipynb
    +-- 📂 dataset
        +-- 📜 dataset.npz
        +-- 📜 ReadMe.pdf
```

### Dodatkowe narzędzia

Dopuszczalne jest korzystanie z bibliotek: `numpy`, `pandas`, `matplotlib`.
Implementacja klasyfikatora powinna być wykonana samodzielnie (bez dodatkowych bibliotek).


### Zadanie

Należy wykonać następujące czynności w celu realizacji niniejszego zadania:
* Wczytaj dane.
* **Zaimplementuj klasyfikator *k* najbliższych sąsiadów (*k*NN)** i uruchom predykcję.
* Opisz jak zmieniają się wyniki klasyfikatora dla różnych wartości *k*, dla różnej liczby klas oraz dla różnej liczby cech.
* Wyświetl tablicę pomyłek (*confusion matrix*). W tym przypadku możesz zastosować gotową implementację z biblioteki `scikit-learn` <https://scikit-learn.org/stable/modules/generated/sklearn.metrics.confusion_matrix.html>.
* Odszukaj przynajmniej dwie inne (niż accuracy) metryki przytatne w klasyfikacji na stronie <https://scikit-learn.org/stable/modules/model_evaluation.html> i opisz ich wyniki. Wytłumacz czym się różnią i co można z nich odczytać.
* Opisz różnicę wyników klasyfikacji obu algorytmów (NN i *k*NN)?


> UWAGA: Wszystkie wykonane elementy zadania powinny posiadać stosowne komentarze i opisy.

### PONIŻEJ WYKONAJ ZADANIE

In [2]:
import numpy as np
import math
from sklearn.metrics import confusion_matrix
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import precision_score

with open('./dataset/dataset.npz', 'rb') as f:
    data = np.load(f)
    train, test = data['train'], data['test']

train_two_classes = train[((train[:,0] == 3) | (train[:,0] == 5))]
test_two_classes = test[((test[:,0] == 3) | (test[:,0] == 5))]

train_five_classes = train[((train[:,0] == 3) | (train[:,0] == 5) | (train[:,0] == 6) | (train[:,0] == 7) | (train[:,0] == 8))]
test_five_classes = test[((test[:,0] == 3) | (test[:,0] == 5) | (test[:,0] == 6) | (test[:,0] == 7) | (test[:,0] == 8))]


class TestSample:

    classified_as = None
    value = None

    def __init__(self, value, classified_as):
        self.value = value
        self.classified_as = classified_as

    def is_properly_classified(self):
        return self.value[0] == self.classified_as

class Neighbour:

    distance = None
    class_value = None

    def __init__(self, class_value, distance):
        self.class_value = class_value
        self.distance = distance

def euclidian_dist(sample1, sample2, features_list):
    value = 0
    for feature in features_list:
        value += (sample1[feature] - sample2[feature]) ** 2
    return math.sqrt(value)

def get_class_from_nearest_neighbours(nearest_neighbours):
    nearest_neighbours_dict = {}
    for nearest_neighbour in nearest_neighbours:
        if nearest_neighbour.class_value in nearest_neighbours_dict:
            nearest_neighbours_dict[nearest_neighbour.class_value].append(nearest_neighbour)
        else:
            nearest_neighbours_dict[nearest_neighbour.class_value] = [nearest_neighbour]
    max_size = 0
    class_value = None
    for k in nearest_neighbours_dict.keys():
        size = len(nearest_neighbours_dict[k])
        if size > max_size:
            max_size = size
            class_value = k
    return class_value


classified_samples = []
def classify_knn_samples(test_set, train_set, k, features_list):
    classified_samples.clear()
    for test_sample in test_set:
        neighbours = []
        for train_sample in train_set:
            dist = euclidian_dist(test_sample, train_sample, features_list)
            neighbours.append(Neighbour(train_sample[0], dist))
        neighbours.sort(key=lambda x: x.distance, reverse=False)
        nearest_neighbours = neighbours[:k]
        classified_class = get_class_from_nearest_neighbours(nearest_neighbours)
        classified_samples.append(TestSample(test_sample, classified_class))

def classify_nn_samples(test_set, train_set, features_list):
    classified_samples.clear()
    for test_sample in test_set:
        min_dist = None
        closest_sample = None
        for train_sample in train_set:
            dist = euclidian_dist(test_sample, train_sample, features_list)
            if min_dist is None or dist < min_dist:
                min_dist = dist
                closest_sample = train_sample
        classified_samples.append(TestSample(test_sample, closest_sample[0]))

def print_accuracy():
    properly_classified_samples = 0
    for classified_sample in classified_samples:
        if classified_sample.is_properly_classified():
            properly_classified_samples += 1
    print("Accuracy is: " + str(properly_classified_samples / len(classified_samples)))

def fill_classification_data(classified, class_true, class_pred):
    for sample in classified:
        class_true.append(sample.value[0])
        class_pred.append(sample.classified_as)

def print_confusion_matrix(classified):
    class_true = []
    class_pred = []
    fill_classification_data(classified, class_true, class_pred)
    print("Confusion matrix: ")
    print(confusion_matrix(class_true, class_pred))

def print_balanced_accuracy(classified):
    class_true = []
    class_pred = []
    fill_classification_data(classified, class_true, class_pred)
    print("Balanced accuracy is: " + str(balanced_accuracy_score(class_true, class_pred)))

def print_precision_score(classified):
    class_true = []
    class_pred = []
    fill_classification_data(classified, class_true, class_pred)
    print("Precision score (macro='weighted') is: " + str(precision_score(class_true, class_pred, average='weighted')))

def print_classification_parameters(classified):
    print_accuracy()
    print_balanced_accuracy(classified)
    print_precision_score(classified)
    print_confusion_matrix(classified)

print("Classes: {3, 5}, features {5, 15}, k 3")
classify_knn_samples(test_two_classes, train_two_classes, 3, [5, 15])
print("kNN:")
print_classification_parameters(classified_samples)
classify_nn_samples(test_two_classes, train_two_classes, [5, 15])
print("NN:")
print_classification_parameters(classified_samples)

# Results:
# For classes {3, 5} features {5, 15} and k equals to 3 we
# have better accuracy for kNN than for simple NN (74% vs 70%)

print('-----------------------------------------')
print("Classes: {3, 5}, features {5, 15}, k 21")
classify_knn_samples(test_two_classes, train_two_classes, 21, [5, 15])
print("kNN:")
print_classification_parameters(classified_samples)
classify_nn_samples(test_two_classes, train_two_classes, [5, 15])
print("NN:")
print_classification_parameters(classified_samples)

# Results:
# For classes {3, 5} features {5, 15} and k equals to 21 we
# have better accuracy for kNN than for simple NN (78% vs 70%) - for bigger k we have better kNN accuracy

print('-----------------------------------------')
print("Classes: {3, 5, 6, 7, 8}, features {5, 15}, k 3")
classify_knn_samples(test_five_classes, train_five_classes, 3, [5, 15])
print("kNN:")
print_classification_parameters(classified_samples)
classify_nn_samples(test_five_classes, train_five_classes, [5, 15])
print("NN:")
print_classification_parameters(classified_samples)

# Results:
# For classes {3, 5, 6, 7, 8} features {5, 15} and k equals to 3 we
# have better accuracy for kNN than for simple NN (42% vs 38%) - for greater number of classes we have worse accuracy for
# both methods

print('-----------------------------------------')
print("Classes: {3, 5, 6, 7, 8}, features {5, 15, 6, 7, 8}, k 3")
classify_knn_samples(test_five_classes, train_five_classes, 3, [5, 15, 6, 7, 8])
print("kNN:")
print_classification_parameters(classified_samples)
classify_nn_samples(test_five_classes, train_five_classes, [5, 15, 6, 7, 8])
print("NN:")
print_classification_parameters(classified_samples)

# Results:
# For classes {3, 5, 6, 7, 8} features {5, 15, 6, 7, 8} and k equals to 3 we
# have better accuracy for kNN than for simple NN (71% vs 69%) - for greater number of features we have better accuracy for both methods

Classes: {3, 5}, features {5, 15}, k 3
kNN:
Accuracy is: 0.7363636363636363
Balanced accuracy is: 0.7363636363636363
Precision score (macro='weighted') is: 0.7364417989417988
Confusion matrix: 
[[41 14]
 [15 40]]
NN:
Accuracy is: 0.7
Balanced accuracy is: 0.7
Precision score (macro='weighted') is: 0.7016666666666667
Confusion matrix: 
[[41 14]
 [19 36]]
-----------------------------------------
Classes: {3, 5}, features {5, 15}, k 21
kNN:
Accuracy is: 0.7818181818181819
Balanced accuracy is: 0.7818181818181819
Precision score (macro='weighted') is: 0.7833167165171152
Confusion matrix: 
[[45 10]
 [14 41]]
NN:
Accuracy is: 0.7
Balanced accuracy is: 0.7
Precision score (macro='weighted') is: 0.7016666666666667
Confusion matrix: 
[[41 14]
 [19 36]]
-----------------------------------------
Classes: {3, 5, 6, 7, 8}, features {5, 15}, k 3
kNN:
Accuracy is: 0.42207792207792205
Balanced accuracy is: 0.43969696969696964
Precision score (macro='weighted') is: 0.4118908580404707
Confusion matrix: